In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn import preprocessing
import xlsxwriter
import time
import warnings
warnings.filterwarnings("ignore")


# Import Spreadsheet
df = pd.ExcelFile('countrydata.xlsx')

#Import sheets to dataframes
aid = df.parse('Foreign Aid').set_index(['Country Name'])
gdp = df.parse('GDP (constant 2010 USD)').set_index(['Country Name'])
vaw = df.parse('Value Added Worker').set_index(['Country Name'])
population = df.parse('Population').set_index(['Country Name'])
poverty = df.parse('Poverty').set_index(['Country Name'])
expectancy = df.parse('Life Expectancy').set_index(['Country Name'])
mortality = df.parse('Mortality').set_index(['Country Name'])
inflation = df.parse('Inflation').set_index(['Country Name'])

start = time.time()


#gdp,mortality,poverty,mortality,poverty,aid,vaw,population,expectancy,inflation

#interpolating values for all variables
for doc in [gdp,mortality,poverty,mortality,poverty,aid,vaw,population,expectancy,inflation]:
    for country in doc.index:
        country_data = doc.loc[country,'1960':'2017']
        x = np.linspace(1,58,58)      #58 years of data per country per variable
        y = np.asarray(country_data.astype(float))
        x [np.argwhere(np.isnan(y))] = np.nan
        x = x[np.logical_not(np.isnan(x))]
        y = y[np.logical_not(np.isnan(y))]
        
        if y.size > 3:


            y = preprocessing.scale(y)


            lineq_params = 0
            exp_params = 0
            log_params = 0

            lineq_params = curve_fit(lambda t,a,b: a*t + b, x, y, p0=[y[0],y[0]])
            exp_params = curve_fit(lambda t,a,b,c,d: a*np.exp(-b*(t-c))+d, x, y, p0=[y[0],y[0],y[0],y[0]],maxfev = 100000)
            log_params = curve_fit(lambda t,a,b: a+b*np.log(t), x, y, p0=[y[0],y[0]], maxfev=100000)

            g = pd.DataFrame(index = x,columns = ['1','2','3'])
            
            #Calculate parameters for linear, exponential and logarithmic curve fit
            for t in g.index:
                g['1'][t] = lineq_params[0][0]*t + lineq_params[0][1]
                g['2'][t] = exp_params[0][0]*np.exp(-exp_params[0][1]*(t-exp_params[0][2])) + exp_params[0][3]
                g['3'][t] = log_params[0][0] + log_params[0][1]*np.log(t)

            #Identify best fit from the 3 equations
            bestindex = -1
            corrmax = -1
            for c in range(1,4):
                q = (np.corrcoef(g[str(c)].astype(float),y))[1][0]
                if q > corrmax:
                    bestindex = c
                    corrmax = q

            if bestindex == 1:
                g = lambda t: lineq_params[0][0]*t + lineq_params[0][1]
            elif bestindex == 2:
                g = lambda t: exp_params[0][0]*np.exp(-exp_params[0][1]*(t-exp_params[0][2])) + exp_params[0][3]
            elif bestindex == 3:
                g = lambda t: log_params[0][0] + log_params[0][1]*np.log(t)
            
            #Identify empty slots and fill with predictions from the best equation data.
            #Use mean and std of available data to unstandardize the data before plugging in
            p = pd.Series(doc.loc[country,'1960':'2017']).isna()   
            std = np.std(doc.loc[country,'1960':'2017'])
            mean = np.mean(doc.loc[country,'1960':'2017'])
            predictions = (((g(np.linspace(1,58,58)) * std) + mean) * p)
            doc.loc[country,'1960':'2017'] = doc.loc[country,'1960':'2017'].fillna(0) + predictions
    print (doc['Indicator Name'][1] + ' done')
print ('All documents done')
end = time.time()

print(end - start)

writer = pd.ExcelWriter('full_data.xlsx', engine='xlsxwriter')
gdp.to_excel(writer,'GDP')
mortality.to_excel(writer,'Mortality')
poverty.to_excel(writer,'Poverty')
aid.to_excel(writer,'Aid')
expectancy.to_excel(writer,'Expectancy')
vaw.to_excel(writer,'Value Added Per Worker')
population.to_excel(writer,'Population')
inflation.to_excel(writer,'Inflation')
writer.save()

GDP (constant 2010 US$) done
Mortality rate, under-5 (per 1,000 live births) done
Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population) done
Mortality rate, under-5 (per 1,000 live births) done
Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population) done
Net official development assistance and official aid received (current US$) done
Services, value added per worker (constant 2010 US$) done
Population, total done
Life expectancy at birth, total (years) done
Inflation, consumer prices (annual %) done
All documents done
128.4631700515747
